# How to perform receiver calibration for EDGES-2

In this demo, we go through how to define and calculate a receiver calibration with `edges`, specifically for EDGES-2. 
For an example, we'll imitate the calibration done for the Bowman+2018 paper. The easiest way to perform that 
calibration *exactly* is via the `edges.alanmode` sub-package. Doing this was covered in the
[alanmode](./using_alnmod_b18.html) tutorial. We will use that as a comparison point in this demo, but show how to 
manually construct your calibration from a set of files, in a more flexible way. Along the way we'll explore the different
components of the calibration and understand how to handle them.

## Data and Imports

We will fetch our data from a Zenodo repo holding the exact raw calibration data used to produce the H2 calibration case in 
Figure 2 of Bowman+2018.

In [2]:
from edges.data import fetch_b18cal_calibrated_s11s, fetch_b18cal_full

In [3]:
full_b18_calobs_path = fetch_b18cal_full()
b18_calibrated_s11_path = fetch_b18cal_calibrated_s11s()

The `fetch_b18cal_full` function downloads all of the raw data for the B18 calibration and aranges it in a directory structure that matches how the data is laid out during a lab-based calibration. This directory structure is understood by some functions in `edges.io`, which helps us build the calibration more easily.

Let's go ahead and perform the default calibration using `alancal` so we have something to compare to later:

In [4]:
# The important function:
from edges.alanmode import alancal

# Classes defining sets of parameters to pass
from edges.alanmode import EdgesScriptParams, Edges2CalobsParams, ACQPlot7aMoonParams

# Functions for reading the outputs of the legacy pipeline
from edges.alanmode import read_modelled_s11s, read_spec_txt, read_specal, LOADMAP, SPEC_LOADMAP

In [5]:
spectra_dir = full_b18_calobs_path / "Spectra"
calobs_default, calibrator_default, *_ = alancal(
    defparams = Edges2CalobsParams(
        s11_path = b18_calibrated_s11_path,
        ambient_acqs = sorted(spectra_dir.glob("Ambient*.acq")),
        hotload_acqs = sorted(spectra_dir.glob("HotLoad*.acq")),
        open_acqs = sorted(spectra_dir.glob("LongCableOpen*.acq")),
        short_acqs = sorted(spectra_dir.glob("LongCableShort*.acq")),
    ),
    acqparams=ACQPlot7aMoonParams.bowman_2018_defaults(),
    calparams=EdgesScriptParams.bowman_2018_defaults(),
)
  

To construct our calibration from scratch, we'll use the following imports:

In [6]:
from edges.io import CalObsDefEDGES2
from edges.cal import CalibrationObservation, Calibrator, S11ModelParams
from edges import modelling as mdl

In [7]:
from astropy import units as un

## Construct a CalibrationObservation from a Standard Directory Layout

To setup all the files we need for a calibration observation, we can use the fact that the directory is in a "standard" layout to construct a file-specification easily:

In [8]:
filespec = CalObsDefEDGES2.from_standard_layout(full_b18_calobs_path)

The `filespec` now has all the files required:

In [9]:
print(filespec.ambient)

LoadDefEDGES2(name='Ambient', thermistor=PosixPath('/home/smurray/.cache/edges/B18-cal-raw-data/Resistance/Ambient_01_2015_300_00_00_00_lab.csv'), s11=LoadS11(calkit=CalkitFileSpec(match=PosixPath('/home/smurray/.cache/edges/B18-cal-raw-data/S11/Ambient01/Match01.s1p'), open=PosixPath('/home/smurray/.cache/edges/B18-cal-raw-data/S11/Ambient01/Open01.s1p'), short=PosixPath('/home/smurray/.cache/edges/B18-cal-raw-data/S11/Ambient01/Short01.s1p')), external=PosixPath('/home/smurray/.cache/edges/B18-cal-raw-data/S11/Ambient01/External01.s1p')), spectra=[PosixPath('/home/smurray/.cache/edges/B18-cal-raw-data/Spectra/Ambient_01_2015_246_02_00_00_lab.acq'), PosixPath('/home/smurray/.cache/edges/B18-cal-raw-data/Spectra/Ambient_01_2015_245_02_00_00_lab.acq')], sparams_file=None)


But this is just a file specification, it doesn't hold actual data. The object that actually holds the calibration data is the 
`CalibrationObservation`. While the `CalibrationObservation` can be constructed directly (without reference to any files -- you 
could even just construct it with simulated values), when we have a file specification like this, it's much easier to construct
the `CalibrationObservation`. 

We do need to choose some parameters for how the data will be read, cut and averaged.
Most importantly, we need to choose the frequency range at which the raw spectra are
cut, as well as the frequency range of the final calibration solutions. These are in 
general different (corresponding to `fstart/fstop` and `wfstart/wfstop` in the
`alancal` function).

In [10]:
calobs = CalibrationObservation.from_edges2_caldef(
    filespec,
    freq_bin_size=8,    # Smooth spectra over 8 bins
    f_low = 50 * un.MHz,
    f_high= 100* un.MHz,
    spectrum_kwargs={
        'default': {
            'f_low': 40.0 * un.MHz,
            'f_high': 110.0 * un.MHz,
            "ignore_times": 7200.0 * un.s,
            "frequency_smoothing": "gauss",
        }
    },
    s11_kwargs={
        "model_params": S11ModelParams(
            model=mdl.Fourier(n_terms=27, transform=mdl.ZerotooneTransform(range=(40, 100)), period=1.5),
            complex_model_type=mdl.ComplexRealImagModel,
            find_model_delay=True,                
        )
    },
    receiver_kwargs = {
        "model_params": S11ModelParams(
            model=mdl.Fourier(n_terms=11, period=1.5, transform= mdl.ZerotooneTransform(range=(40,100))),
            complex_model_type=mdl.ComplexRealImagModel,
            find_model_delay=True,
        )
    },
    loss_model_params = S11ModelParams(
        model=mdl.Fourier(n_terms=27, period=1.5, transform=mdl.ZerotooneTransform(range=(40,100))),
        complex_model_type=mdl.ComplexRealImagModel,
    ),
)

Reading Ambient_01_2015_246_02_00_00_lab.acq: 680lines [00:00, 852.02lines/s]
Reading Ambient_01_2015_245_02_00_00_lab.acq: 605lines [00:00, 837.62lines/s]/data7/smurray/edges/edges-analysis/.venv/lib/python3.11/site-packages/read_acq/read_acq.py:332: UserWarning: nspec and length of spectrum do not match
  warnings.warn(str(e), stacklevel=1)
Reading Ambient_01_2015_245_02_00_00_lab.acq: 637lines [00:00, 844.52lines/s]
Reading HotLoad_01_2015_247_00_00_00_lab.acq: 752lines [00:00, 845.35lines/s]
Reading HotLoad_01_2015_246_04_00_00_lab.acq: 5624lines [00:06, 830.90lines/s]
Reading LongCableOpen_01_2015_245_00_00_00_lab.acq: 464lines [00:00, 833.70lines/s]
Reading LongCableOpen_01_2015_244_00_00_00_lab.acq: 6836lines [00:08, 836.36lines/s]
Reading LongCableOpen_01_2015_243_14_00_00_lab.acq: 2582lines [00:03, 837.43lines/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

The `CalibrationObservation` does not have the calibration solutions in it -- only the data required to generate those solutions. Different 
methods can be used to compute the final calibration solutions from this data. The iterative method used in B18 can be used:

In [ ]:
from edges.cal import get_calcoeffs_iterative

This function takes the calibration observation, along with some parameters that determine how it runs, and ultimately returns a `Calibrator` object, which simply holds the calibration solutions. Let's ensure that the parameters here match those used in the legacy pipeline:

In [ ]:
calibrator = get_calcoeffs_iterative(
    calobs, 
    cterms=6, 
    wterms=5, 
    apply_loss_to_true_temp=False, 
    smooth_scale_offset_within_loop=False, 
    cable_delay_sweep=np.arange(0, -1e-8, -1e-9),
    ncal_iter=8,
    scale_offset_poly_spacing=0.5,
    t_load_guess=300*un.K,
    t_load_ns_guess=1000.0*un.K
)